# Quickstart

This short tutorial will teach you how to consume senpy services for several tasks, and how to take advantage of the features of the framework.

In particular, it covers:

* Annotating text with sentiment and emotion using interoperable services
* Switching to different services (service interoperability)
* Getting results in different formats (Turtle, XML, text...)
* Asking for specific emotion models (automatic model conversion)
* Listing available services in an endpoint
* Calling multiple services in the same request (Pipelines)

These topics are split into two separate tutorials.

Reading all the sections is not necessary, although it is encouraged in order to get a glimpse of all the features.

* [Senpy in 1 minute](./Quickstart-1minute.ipynb) shows how to query the API.
* [Senpy in 10 minutes](./Quickstart-10minutes.ipynb) introduces basic sentiment and emotion analysis.
* [Senpy in 30 minutes](./Quickstart-30minutes.ipynb) builds on the previous and adds more advanced functionalities, such as emotion conversion, field selection and pipelines.